In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
import random

In [2]:
tabla_completa = pd.read_csv('events_up_to_01062018.csv');
tabla_completa.info()

C:\Users\Martín-\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341681 entries, 0 to 2341680
Data columns (total 23 columns):
timestamp                   object
event                       object
person                      object
url                         object
sku                         float64
model                       object
condition                   object
storage                     object
color                       object
skus                        object
search_term                 object
staticpage                  object
campaign_source             object
search_engine               object
channel                     object
new_vs_returning            object
city                        object
region                      object
country                     object
device_type                 object
screen_resolution           object
operating_system_version    object
browser_version             object
dtypes: float64(1), object(22)
memory usage: 410.9+ MB


In [3]:
res = pd.read_csv('labels_training_set.csv')

In [4]:
tabla_completa['aux']=1
#Voy a contar la cantidad para cada tipo de evento de cada persona
num_eventos = tabla_completa.groupby(['person','event'])['aux'].sum()

num_eventos = num_eventos.unstack().reset_index().fillna(0)
num_eventos = num_eventos.set_index('person')
num_eventos['total_eventos'] = num_eventos.sum(axis=1)
#normalizo los numeros obtenidos según la cantidad de eventos totales de esa persona
num_eventos_norm = num_eventos.loc[:,"ad campaign hit":"visited site"].div(num_eventos["total_eventos"], axis=0)
#num_eventos.sort_values(by='conversion',ascending=False)
#num_eventos_norm.sort_values(by='conversion',ascending=False)

In [5]:
tabla_completa['timestamp'] = pd.to_datetime(tabla_completa['timestamp'])
#tabla_completa['timestamp'].max() -tabla_completa['timestamp'].min()

time_for_user = tabla_completa.groupby(['person'])['timestamp'].apply(lambda x: x.max() - x.min())
time_for_user = time_for_user.reset_index()
time_for_user['total_horas'] = time_for_user['timestamp'].dt.total_seconds()/3600
time_for_user = time_for_user.drop(['timestamp'],axis='columns')

In [6]:
num_eventos_norm = num_eventos_norm.reset_index()
num_eventos = num_eventos.reset_index()
total_eventos = pd.merge(num_eventos,num_eventos_norm, on='person',how='inner')
total_eventos = pd.merge(total_eventos,time_for_user,on='person',how='inner')

In [7]:
intento = total_eventos.drop(['conversion_x','conversion_y'],axis='columns')
intento_f = pd.merge(intento,res,on='person',how='inner')
intento_f = intento_f.set_index('person')
intento_f.head()

,ad campaign hit_x,brand listing_x,checkout_x,generic listing_x,lead_x,search engine hit_x,searched products_x,staticpage_x,viewed product_x,visited site_x,...,checkout_y,generic listing_y,lead_y,search engine hit_y,searched products_y,staticpage_y,viewed product_y,visited site_y,total_horas,label
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.500000,0.166667,0.0,0.000000,0.000000,0.0,0.000000,0.333333,4.013889,0
000c79fe,1.0,0.0,1.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0,...,0.058824,0.058824,0.0,0.058824,0.529412,0.0,0.176471,0.058824,0.172222,0
001802e4,5.0,0.0,1.0,4.0,0.0,0.0,4.0,0.0,4.0,1.0,...,0.052632,0.210526,0.0,0.000000,0.210526,0.0,0.210526,0.052632,0.179444,0
0019e639,29.0,165.0,15.0,28.0,0.0,13.0,11.0,0.0,189.0,19.0,...,0.031847,0.059448,0.0,0.027601,0.023355,0.0,0.401274,0.040340,2743.832500,0
001b0bf9,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.285714,0.142857,0.0,0.000000,0.000000,0.0,0.285714,0.000000,0.255000,0


In [8]:
x, y = intento_f.iloc[:,:-1],intento_f.iloc[:,-1]

In [9]:
test_size = 0.2
random_state = 0

In [10]:
datos_train, datos_test, cat_train, cat_test = train_test_split(x, y, test_size = test_size, random_state = random_state)

In [11]:
clf = RandomForestClassifier(n_jobs=1000,random_state=0)

In [12]:
clf.fit(datos_train,cat_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1000,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [13]:
cat_pred = clf.predict(datos_test)

print('accuracy: {0:.2f}%'.format(accuracy_score(cat_test,cat_pred)*100))

accuracy: 94.36%


In [14]:
rmse = np.sqrt(mean_squared_error(cat_test, cat_pred))
print("RMSE: %f" % (rmse))

RMSE: 0.237486


In [15]:
final = pd.read_csv('trocafone_kaggle_test.csv')
final = pd.merge(intento,final,on='person',how='inner')
final = final.set_index('person')
x_entrega = final

In [16]:
entrega = clf.predict(x_entrega)
y_summit = pd.DataFrame(entrega)
y_summit.columns = ['label']
y_summit['person'] = x_entrega.reset_index()['person']
y_person = x_entrega.reset_index()['person']

y_summit = y_summit[['person','label']]
y_summit.to_csv('summit12.csv', index=False)

In [17]:
y_summit.max()

person    fffd1246
label            1
dtype: object